# **Tutorial 2** - scientific Python ecosystem: `pandas` and `GeoPandas`

In this tutorial we will learn using
 1. [**pandas**](https://pandas.pydata.org/docs/): **tabular data** stored in ``pandas.DataFrame``s
 2. [**GeoPandas**](https://geopandas.org/en/stable/docs.html): **aspatial data** stored in ``geopandas.GeoDataFrame``s

within [`PyGMT`](https://www.pygmt.org) to create histograms and different maps.

-----
This tutorial is part of the AGU2024 annual meeting GMT/PyGMT pre-conference workshop (PREWS9) **Mastering Geospatial Visualizations with GMT/PyGMT**
- Conference: https://agu.confex.com/agu/agu24/meetingapp.cgi/Session/226736
- GitHub: https://github.com/GenericMappingTools/agu24workshop
- Website: https://www.generic-mapping-tools.org/agu24workshop
- Recommended version: PyGMT v0.13.0 with GMT 6.5.0

## 0. General stuff

Import the required packages, besides [`PyGMT`](https://www.pygmt.org) itself, we use [`pandas`](https://pandas.pydata.org/docs/) and [`GeoPandas`](https://geopandas.org/en/stable/docs.html):

In [ ]:
import pygmt

import pandas as pd
import geopandas as gpd

# Use a resolution of only 150 dpi for the images within the Jupyter notebook, to keep the file small
img_dpi = 150

## 1. `pandas`

### 1.1 Tabular data - `pandas.DataFrame`

Use an example dataset with tabular data provided by `PyGMT` and load it into a `pandas.DataFrame`. This dataset contains earthquakes in the area of Japan.

In [ ]:
df_jp_eqs = pygmt.datasets.load_sample_data(name="japan_quakes")
df_jp_eqs.head()

### 1.2 Create a Cartesian histogram

First we create a Cartesian histogram for the moment magnitude distribution of all earthquakes in the dataset.

In [ ]:
mag_min = df_jp_eqs["magnitude"].min(); print(mag_min)
mag_max = df_jp_eqs["magnitude"].max(); print(mag_max)

In [ ]:
# Choose the bin width of the bars; feel free to play around with this value
step_histo = 0.2

fig = pygmt.Figure()

fig.histogram(
    data=df_jp_eqs["magnitude"],
    projection="X10c",
    # Determine y range automatically
    region=[mag_min - step_histo, mag_max + step_histo * 2, 0, 0],
    # Define the frame, add labels to the x-axis and y-axis
    frame=["WSne", "x+lmoment magnitude", "y+lcounts"],
    # Generate evenly spaced bins
    series=step_histo,
    # Fill bars with color "orange"
    fill="orange",
    # Draw gray outlines with a width of 1 point
    pen="1p,gray",
    # Choose histogram type 0, i.e., counts [Default]
    histtype=0,
)

fig.show(dpi=img_dpi)

Use code example above as orientation, and create a similar histogram showing the hypocentral depth distribution for all earthquakes in the dataset.

In [ ]:
# Your code (:

For details on creating Cartesian histograms you may find the tutorial [**Cartesian histograms**](https://www.pygmt.org/v0.13.0/tutorials/advanced/cartesian_histograms.html) helpful.

### 1.3 Create a geographical map of the earthquakes

Now it's time to create a geographical map showing the earthquakes. You can start with using [`pygmt.Figure.coast`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.Figure.coast.html) and [`pygmt.Figure.plot`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.Figure.plot.html). For plotting the earthquakes as size (moment magnitude) and color (hypocentral depth) coded circles on top of the map please follow the tutorial [**Plotting data points**](https://www.pygmt.org/v0.13.0/tutorials/basics/plot.html).

In [ ]:
# Your code (:

## 2. `GeoPandas`

### 2.1 Line geometry

Features which can be represented as a **line geometry** are for example rivers, roads, national boundaries, shorlines, and any kind of profiles.

#### 2.1.1 Aspatial Data - `geopandas.GeoDataFrame` with line geometry

First we download some data into in a `geopandas.GeoDataFrame`. This dataset contains European rivers with its length and name.

In [ ]:
gpd_rivers_org = gpd.read_file(
    "https://www.eea.europa.eu/data-and-maps/data/wise-large-rivers-and-large-lakes/" + \
    "zipped-shapefile-with-wise-large-rivers-vector-line/zipped-shapefile-with-wise-large-rivers-vector-line/" + \
    "at_download/file/wise_large_rivers.zip"
)
gpd_rivers_org.head()

Have a look at the values in the geometry column. The coordinates are currently not given in the geographic coordinate reference system (longitude/latitude) and have to be converted. This can be done directly with `GeoPandas`.

In [ ]:
gpd_rivers_org.crs
gpd_rivers = gpd_rivers_org.to_crs('EPSG:4326')
gpd_rivers.head()

#### 2.1.2 Create a geographical map of the rivers

Let's plot the rivers represented on top of a map. The `geopandas.DataFrame` can be directly passed to the `data` parameter of [`pygmt.Figure.plot`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.Figure.plot.html). Use the `pen` parameter to adjust the lines. The string argument has the form "*width*,*color*,*style*". Different line styles are explained in the Gallery example [**Line styles**](https://www.pygmt.org/v0.13.0/gallery/lines/linestyles.html).

In [ ]:
fig = pygmt.Figure()

fig.coast(
    projection="M10c", 
    region=[-10, 30, 35, 57],
    land="gray99",
    shorelines="1/0.1p,gray50",
    frame=True,
)

fig.plot(data=gpd_rivers, pen="0.5p,steelblue,solid")

fig.show(dpi=img_dpi)

#### 2.1.3 Plot subsets of the rivers differently

Now we want to filter the dataset based on the river length and plot the subsets differently.

In [ ]:
fig = pygmt.Figure()

fig.coast(
    projection="M10c", 
    region=[-10, 35, 35, 58],
    land="gray99",
    shorelines="1/0.1p,gray50",
    frame=True,
)

# Split the dataset into two subsets of shorter and longer rivers
# Feel free to play around with the limit
len_limit = 700000  # in meters
gpd_rivers_short = gpd_rivers[gpd_rivers["Shape_Leng"] < len_limit]
gpd_rivers_long = gpd_rivers[gpd_rivers["Shape_Leng"] > len_limit]

# Add a heading (+H) to the legend and adjust the font size (+f) to 9 points
fig.plot(data=gpd_rivers_short, pen="0.5p,orange", label=f"shorter {len_limit} m+Hriver length+f9p")
fig.plot(data=gpd_rivers_long, pen="0.5p,darkred", label=f"longer {len_limit} m")

fig.legend(position="jLT+o0.1c", box="+gwhite@30+p0.1p,gray50")

fig.show(dpi=img_dpi)

#### 2.1.4 Plot the rivers with color-coding for the river length

Use the gallery example [**Line colors with a custom CPT**](https://www.pygmt.org/v0.13.0/gallery/lines/line_custom_cpt.html) to plot the rivers with color-coding for the river length.

In [ ]:
# Your code (:

### 2.2 Polygon geometry

Any kind of areas, like continents, countries, and states can be stored as **polygon geometry**.

#### 2.2.1 Aspatial Data - `geopandas.GeoDataFrame` with polygon geometry

Again we download some data into in a `geopandas.GeoDataFrame`. This dataset contains information regarding airbnb rentals, socioeconomics, and crime in Chicagos.
This time we are lucky and the data is directly provided in the geographic coordinate reference system (longitude/latitude) and no further coordinate transformation is needed.

In [ ]:
gdf_airbnb = gpd.read_file("https://geodacenter.github.io/data-and-lab/data/airbnb.zip")
gdf_airbnb.head()

#### 2.2.2 Create a choropleth map

Here we are going to create a so-called **choropleth map**. Such a visualization is helpful to show a quantity which vary between subregions of a study area, e.g., countries or states. The dataset contains several columns, here we will focus on the column ``"population"``, but feel free to modify the code below and use another quantity for the color-coding!

In [ ]:
popul_min = gdf_airbnb["population"].min(); print(popul_min)
popul_max = gdf_airbnb["population"].max(); print(popul_max)

In [ ]:
fig = pygmt.Figure()

# frame=True adds a frame using the default settings
# Use "rltb" (right, left, top, bottom) to plot no frame at all which
# can be useful when inserting the figure within a document
fig.basemap(region=[-88, -87.5, 41.62, 42.05], projection="M10c", frame=True)

# Set up colormap
pygmt.makecpt(cmap="SCM/bilbao", series=[popul_min, popul_max, 10])
# Add vertical colorbar at left side
fig.colorbar(frame="x+lpopulation", position="jLM+v")

# Plot the polygons with color-coding for the population
fig.plot(
    data=gdf_airbnb, 
    pen="0.2p,gray10", 
    fill="+z", 
    cmap=True,
    aspatial="Z=population",
)

fig.show(dpi=img_dpi)

## 3. Additional comments

Some helpful and interesting aspects:

- Use suitable colormaps for your data: [**Scientific colourmaps by Fabio Crameri**](https://www.fabiocrameri.ch/colourmaps/), see also the publications [Crameri et al. 2024](https://doi.org/10.1002/cpz1.1126) and [Crameri et al. 2020](https://doi.org/10.1038/s41467-020-19160-7)
- Datasets provided by `GeoPandas`: Checkout the [**geodatasets**](https://geodatasets.readthedocs.io) libaray
- Convert other objects to `pandas` or `GeoPandas` objects to make them usable in `PyGMT`: For example, convert [**OSMnx**](https://osmnx.readthedocs.io)'s `MultiDiGraph` to a `geopandas.DataFrame`
- Create more complex geometries: Combine `GeoPandas` with [**shapely**](https://shapely.readthedocs.io) (i.e., `from shapely.geometry import Polygon`)
- Support of OGR formats: Use [`geopandas.read_file`](https://geopandas.org/en/v1.0.1/docs/reference/api/geopandas.read_file.html) to load data provided as shapefile (.shp), GeoJSON (.geojson), geopackage (.gpkg), etc.

## 4. Orientation / suggestion for "2.1.4 Plot the rivers with color-coding for the river length"

Below you find a rough code shipset for plotting the rives with color-coding in section 2.1.4.

In [ ]:
fig = pygmt.Figure()

fig.coast(
    projection="M10c", 
    region=[-10, 35, 35, 58],
    land="gray99",
    shorelines="1/0.1p,gray50",
    frame=True,
)

pygmt.makecpt(cmap="SCM/oslo",series=[gpd_rivers.Shape_Leng.min(), 1500000], reverse=True)
fig.colorbar(frame=["x+lriver length", "y+lm"], position="+ef0.2c")

for i_river in range(len(gpd_rivers)):
    fig.plot(
        data=gpd_rivers[gpd_rivers.index==i_river],
        zvalue=gpd_rivers.loc[i_river, "Shape_Leng"],
        pen="0.5p",
        cmap=True,
    )

fig.show(dpi=img_dpi)